# **Seri Bubble Sort**

In [ ]:
import time
import random

class Timer:
    def start(self):
        self.start_time = time.perf_counter()

    def stop(self):
        self.stop_time = time.perf_counter()
        elapsed_time = (self.stop_time - self.start_time) * 1000
        print(f"{elapsed_time:.2f} ms'de tamamlandı.")

def bubble_sort(arr):
    n = len(arr)
    # Tüm elemanlar sıralanana kadar döngüyü çalıştır
    for i in range(n):
        swapped = False  # Bayrak, herhangi bir yer değiştirme işlemi yapıldığını belirtir
        # Her bir geçişte, son i eleman zaten sıralı olduğu için onları kontrol etme
        for j in range(0, n-i-1):
            # Elemanlar sırasıyla karşılaştırılıp yer değiştirilir
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
                swapped = True  # Yer değiştirme yapıldı
        # Herhangi bir yer değiştirme yapılmadıysa, dizi zaten sıralıdır ve döngüyü sonlandır
        if not swapped:
            break
    return arr

# Zamanlayıcıyı başlat
timer = Timer()

# 10,000 elemanlı bir dizi oluştur
arr = [random.randint(1, 100000) for _ in range(10000)]

# Bubble Sort'u çalıştır ve süreyi ölç
timer.start()
sorted_arr = bubble_sort(arr)
timer.stop()

print("Sıralanmış dizi:", sorted_arr[:100])  # Dizi uzun olacağı için ilk 100 elemanı yazdırıyoruz.


## Ardışık paralel Bubble Sort

In [ ]:
import concurrent.futures
import random
import time

class Timer:
    def start(self):
        self.start_time = time.perf_counter()

    def stop(self):
        self.stop_time = time.perf_counter()
        elapsed_time = (self.stop_time - self.start_time) * 1000
        return elapsed_time

def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        swapped = False
        for j in range(0, n - i - 1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
                swapped = True
        if not swapped:
            break
    return arr

def merge_sorted_chunks(chunks):
    merged = []
    while chunks:
        min_val = min(chunk[0] for chunk in chunks if chunk)
        for chunk in chunks:
            if chunk and chunk[0] == min_val:
                merged.append(chunk.pop(0))
                if not chunk:
                    chunks.remove(chunk)
                break
    return merged

def parallel_bubble_sort(arr, num_chunks):
    n = len(arr)
    chunk_size = n // num_chunks
    chunks = [arr[i:i + chunk_size] for i in range(0, n, chunk_size)]

    # Paralel sıralama işlemi zamanlaması
    parallel_timer = Timer()
    parallel_timer.start()

    with concurrent.futures.ProcessPoolExecutor(max_workers=num_chunks) as executor:
        futures = [executor.submit(bubble_sort, chunk) for chunk in chunks]
        results = [future.result() for future in futures]

    parallel_time = parallel_timer.stop()

    # Seri birleştirme işlemi zamanlaması
    sequential_timer = Timer()
    sequential_timer.start()

    sorted_arr = merge_sorted_chunks(results)

    sequential_time = sequential_timer.stop()

    return sorted_arr, parallel_time, sequential_time

# Rastgele dizi oluşturma (10,000 eleman)
arr = [random.randint(1, 100000) for _ in range(10000)]

# Dizi 4 parçaya bölünecek
num_chunks = 4

# Paralel Bubble Sort'u çalıştırma
sorted_arr, parallel_time, sequential_time = parallel_bubble_sort(arr, num_chunks)

# Süreleri yazdırma
print(f"Paralel kısım süresi: {parallel_time:.2f} ms")
print(f"Seri kısım (merge) süresi: {sequential_time:.2f} ms")

# Hız oranını hesaplama
if parallel_time > 0:  # ZeroDivisionError kontrolü
    speedup_ratio = parallel_time / sequential_time
    print(f"Seri kısmın paralel kısma göre hız oranı: {speedup_ratio:.4f}")
else:
    print("Paralel kısım süresi 0 olduğundan hız oranı hesaplanamadı.")
parallel_time
# Toplam süre
total_time = parallel_time + sequential_time
print(f"Toplam paralel sıralama süresi: {total_time:.2f} ms")

print("Sıralanmış dizi:", sorted_arr[:500])  # Dizi uzun olacağı için ilk 500 elemanı yazdırıyoruz.
